# Assignment 1
### Armin Danesh | adaneshp
----------------

First we need to import the necessary python libraries.

In [8]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

Num GPUs Available:  2


Before doing anything, we set the seed for our randome because we want to make sure that we get the same randomness evertime so that we can analyze our algoithms without changing our dataset.

In [9]:
np.random.seed(713)
tf.random.set_seed(137)

Now we want to load the data and split it according to our needs. In order to do that first we put the data inside a data frame from pandas:

In [10]:
names = os.listdir ('./train')

files = []
annos = []

for name in names:
    files.append(name)
    annos.append(name[:3])

df = pd.DataFrame({'file': files, 'anno': annos})

df.head()

,file,anno
0,cat.797.jpg,cat
1,dog.5704.jpg,dog
2,cat.10234.jpg,cat
3,dog.6027.jpg,dog
4,cat.7413.jpg,cat


Now that we have the list we can easily use the split from sklearn:

In [11]:
train_set, val_set = train_test_split(df, test_size=0.3, random_state=713)
train_set.shape, val_set.shape

((17500, 2), (7500, 2))

Now that we our sets splitted we can do augmentation and pre process the data. We also batch the files for faster training and testing, and for better utilization of the GPUs:

In [12]:
train_transform = tf.keras.preprocessing.image.ImageDataGenerator (rotation_range=10, rescale=1./255, shear_range=0.1, zoom_range=0.2, horizontal_flip=True, vertical_flip=True)
val_transform = tf.keras.preprocessing.image.ImageDataGenerator (rescale=1./255)

train_data = train_transform.flow_from_dataframe(dataframe=train_set, directory='./train/', x_col='file', y_col='anno', target_size=(200, 200), class_mode='categorical', batch_size=32)

val_data = val_transform.flow_from_dataframe(dataframe=val_set, directory='./train/', x_col='file', y_col='anno', target_size=(200, 200), class_mode='categorical', batch_size=32)


Found 17500 validated image filenames belonging to 2 classes.
Found 7500 validated image filenames belonging to 2 classes.


In [17]:
train_data[1][1]

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

Now that we have the augmented data ready for training, for transfer learning we should bring the model that we like. We also have to make sure that we freez the model as we want to do transfer learning. In the following code we bring the model that we like, with the weights that are trained over imagenet, giving the input image size that our transform function generates, and freezing all the trainable paramteres.

In [ ]:
model_vgg19 = tf.keras.applications.VGG19(weights="imagenet", include_top=False, input_shape=(200, 200, 3))
model_vgg19.trainable = False
model_vgg19.summary()

We loaded our model topless. As we are using it for our backbone, and we want categorization, after the last layer we need to flatten the results, move to two outputs using a fully connected layer. On top of all with a softmax activation layer, we will get probabilities:

In [ ]:
final_model = tf.keras.models.Sequential()
final_model.add(model_vgg19)
final_model.add(tf.keras.layers.Flatten())
final_model.add(tf.keras.layers.Dense(128, activation='relu'))
final_model.add(tf.keras.layers.Dense(64, activation='relu'))
final_model.add(tf.keras.layers.Dense(32, activation='relu'))
final_model.add(tf.keras.layers.Dense(2, activation='softmax'))

The model architectur is completely ready to use. Now we have to choose an optimizer, a learning rate, if wanted a momentum, our loss function, and our preferred metric:

In [ ]:
chosen_optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Now we fit the model to our training data. Please note that since we are using transfer learning, we do not touch the trained weights.

In [ ]:
BATCH_SIZE = 32

VGG19_Freezed = tf.keras.callbacks.TensorBoard(log_dir="VGG19_Freezed", histogram_freq=1)
final_model.fit(train_data, epochs=3, validation_data=val_data, validation_steps=val_set.shape[0]//BATCH_SIZE, steps_per_epoch=train_set.shape[0]//BATCH_SIZE, callbacks=VGG19_Freezed)

We can observer the results using tensorboard. It gives us all the useful information. Now for fine tuning, we unfreeze the VGG layer and run the training again.

In [ ]:
final_model.layers[0].trainable = True
final_model.summary()
final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(train_data, epochs=10, validation_data=val_data, validation_steps=val_set.shape[0]//BATCH_SIZE, steps_per_epoch=train_set.shape[0]//BATCH_SIZE, callbacks=tf.keras.callbacks.TensorBoard(log_dir="VGG19_Unfreezed", histogram_freq=1))

For looking which model is working better, we change the model to mobilnetv2 and run the same experiments without any specific change. First the freezed version:

In [ ]:
model_mobilenetv2 = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=False, input_shape=(200, 200, 3))
model_mobilenetv2.trainable = False
model_mobilenetv2.summary()

final_model = tf.keras.models.Sequential()
final_model.add(model_mobilenetv2)
final_model.add(tf.keras.layers.Flatten())
final_model.add(tf.keras.layers.Dense(128, activation='relu'))
final_model.add(tf.keras.layers.Dense(64, activation='relu'))
final_model.add(tf.keras.layers.Dense(32, activation='relu'))
final_model.add(tf.keras.layers.Dense(2, activation='softmax'))

final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(train_data, epochs=3, validation_data=val_data, validation_steps=val_set.shape[0]//BATCH_SIZE, steps_per_epoch=train_set.shape[0]//BATCH_SIZE, callbacks=tf.keras.callbacks.TensorBoard(log_dir="MobileNetV2_Freezed", histogram_freq=1))

Then, same as before, we change unfreeze mobilenet for fine tuning:

In [ ]:
final_model.layers[0].trainable = True
final_model.summary()
final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(train_data, epochs=10, validation_data=val_data, validation_steps=val_set.shape[0]//BATCH_SIZE, steps_per_epoch=train_set.shape[0]//BATCH_SIZE, callbacks=tf.keras.callbacks.TensorBoard(log_dir="MobileNetV2_Unfreezed", histogram_freq=1))

Same process goes for ResNet. First freezed:

In [ ]:
model_resnet50 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=(200, 200, 3))
model_resnet50.trainable = False
model_resnet50.summary()

chosen_optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
BATCH_SIZE = 32

final_model = tf.keras.models.Sequential()
final_model.add(model_resnet50)
final_model.add(tf.keras.layers.Flatten())
final_model.add(tf.keras.layers.Dense(128, activation='relu'))
final_model.add(tf.keras.layers.Dense(64, activation='relu'))
final_model.add(tf.keras.layers.Dense(32, activation='relu'))
final_model.add(tf.keras.layers.Dense(2, activation='softmax'))

final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(train_data, epochs=3, validation_data=val_data, validation_steps=val_set.shape[0]//BATCH_SIZE, steps_per_epoch=train_set.shape[0]//BATCH_SIZE, callbacks=tf.keras.callbacks.TensorBoard(log_dir="ResNet50_Freezed", histogram_freq=1))

And again the unfreezed version:

In [ ]:
final_model.layers[0].trainable = True
final_model.summary()
final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(train_data, epochs=10, validation_data=val_data, validation_steps=val_set.shape[0]//BATCH_SIZE, steps_per_epoch=train_set.shape[0]//BATCH_SIZE, callbacks=tf.keras.callbacks.TensorBoard(log_dir="ResNet50_Unfreezed", histogram_freq=1))

------------------

In this part instead of using available loaders, we write our own function to do loading and tranforms for us. We bring the same packages from previous section and set the randomness.

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn
import cv2
import random

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

np.random.seed(713)
tf.random.set_seed(137)

Num GPUs Available:  2


Now we start writing the ImageLoader function. For readability, each section is commented:

In [2]:
def ImageLoader(input_path):
    # Just like before, we get the names of the files in a dataframe
    names = os.listdir (input_path)
    files = []
    annos = []
    for name in names:
        files.append(name)
        annos.append(name[:3])

    df = pd.DataFrame({'file': files, 'anno': annos})
    # Splitting the data into training and validation sets
    train_set, val_set = train_test_split(df, test_size=0.3, random_state=713)
    train_set = train_set.reset_index()  # make sure indexes pair with number of rows
    val_set = val_set.reset_index()  # make sure indexes pair with number of rows

    train_set.head()
    train_imgs = []
    train_labels = []
    # Loading each image and transforming it
    for index, row in train_set.iterrows():
        image = cv2.imread(input_path+row['file'])
        rows, cols, ch = image.shape

        # Resizing image
        image = cv2.resize(image, (200, 200))
        rows, cols, ch = image.shape
        # cv2.imwrite('resize.jpg', image)

        # Rotating the image
        M = cv2.getRotationMatrix2D(((cols-1)/2.0,(rows-1)/2.0),random.randint(0, 180), 1)
        image = cv2.warpAffine(image, M, (cols,rows))
        rows, cols, ch = image.shape
        # cv2.imwrite('rotate.jpg', image)

        # Affine transform image
        pts1 = np.float32([[random.randint(30, 50),random.randint(30, 50)],[random.randint(180, 200),random.randint(30, 50)],[random.randint(30, 50),random.randint(180, 200)]])
        pts2 = np.float32([[random.randint(10, 30),random.randint(10, 30)],[random.randint(150, 180),random.randint(10, 30)],[random.randint(10, 30),random.randint(150, 180)]])
        M = cv2.getAffineTransform(pts1,pts2)
        image = cv2.warpAffine(image,M,(cols,rows))
        rows, cols, ch = image.shape
        # cv2.imwrite('affine.jpg', image)

        # Perspective transform image
        pts1 = np.float32([[random.randint(30, 50),random.randint(30, 50)],[random.randint(180, 200),random.randint(30, 50)],[random.randint(180, 200),random.randint(180, 200)],[random.randint(30, 50),random.randint(180, 200)]])
        pts2 = np.float32([[random.randint(10, 30),random.randint(10, 30)],[random.randint(160, 180),random.randint(10, 30)],[random.randint(160, 180),random.randint(160, 180)],[random.randint(10, 30),random.randint(160, 180)]])
        M = cv2.getPerspectiveTransform(pts1,pts2)
        image = cv2.warpPerspective(image,M,(200,200))
        rows, cols, ch = image.shape
        # cv2.imwrite('perspective.jpg', image)

        image = image/255. # This is for normalization
        
        train_imgs.append(image)
        train_labels.append(row['anno'])
        # break

    val_imgs = []
    val_labels = []

    # Same happens for validation images, however we just resize the images.
    for index, row in val_set.iterrows():
        image = cv2.imread(input_path+row['file'])
        rows, cols, ch = image.shape

        image = cv2.resize(image, (200, 200))
        rows, cols, ch = image.shape
        # break
    # cv2.imwrite('resize.jpg', image)

        image = image/255. #This is for normalization

        val_imgs.append(image)
        val_labels.append(row['anno'])

    # The model accepts np arrays and not lists. Thus we convert the lists we created.
    train_imgs = np.array(train_imgs)
    train_labels = np.array(train_labels)
    train_labels_coded = []

    val_imgs = np.array(val_imgs)
    val_labels = np.array(val_labels)
    val_labels_coded = []

    # Preparing a one hot encoding for our labels
    for label in train_labels:
        print (label)
        # break
        if label == "dog":
            train_labels_coded.append([0.,1.])
        elif label == "cat":
            train_labels_coded.append([1.,0.])
    
    for label in val_labels:
        if label == "dog":
            val_labels_coded.append([0.,1.])
        elif label == "cat":
            val_labels_coded.append([1.,0.])
    
    train_labels_coded = np.array(train_labels_coded)
    val_labels_coded = np.array(val_labels_coded)
    

    return train_imgs, train_labels_coded, val_imgs, val_labels_coded



Now, simply by calling our function, we get all the inputs for the model:

In [3]:
training_imgs, training_labels, validation_imgs, validation_labels = ImageLoader('./train/')
training_imgs.shape, training_labels.shape, validation_imgs.shape, validation_labels.shape

dog
cat
cat
cat
dog
cat
dog
cat
dog
dog
cat
dog
cat
cat
dog
cat
dog
cat
cat
dog
cat
cat
cat
cat
dog
dog
cat
cat
cat
dog
cat
cat
cat
dog
dog
cat
dog
dog
cat
cat
cat
cat
dog
dog
dog
cat
cat
dog
cat
cat
cat
cat
dog
cat
dog
cat
dog
dog
dog
dog
dog
cat
dog
dog
cat
dog
cat
cat
dog
dog
cat
cat
cat
cat
dog
dog
cat
dog
dog
dog
cat
dog
cat
cat
cat
dog
cat
cat
cat
dog
dog
cat
dog
cat
dog
dog
dog
cat
dog
dog
cat
cat
cat
cat
dog
dog
cat
dog
cat
dog
dog
dog
cat
cat
cat
dog
cat
dog
dog
dog
cat
dog
dog
cat
cat
cat
dog
cat
dog
cat
dog
cat
dog
cat
cat
dog
cat
dog
dog
dog
dog
dog
cat
dog
cat
dog
cat
dog
dog
dog
cat
cat
cat
dog
dog
dog
cat
dog
dog
dog
cat
cat
dog
dog
cat
cat
dog
dog
cat
cat
dog
cat
dog
dog
cat
cat
dog
dog
dog
cat
cat
dog
dog
cat
dog
cat
dog
cat
dog
cat
cat
dog
cat
dog
cat
dog
cat
dog
cat
cat
cat
cat
dog
cat
cat
dog
dog
cat
cat
dog
dog
dog
cat
dog
dog
cat
cat
dog
cat
dog
dog
dog
dog
dog
dog
dog
dog
dog
cat
dog
dog
dog
cat
dog
dog
cat
cat
cat
dog
cat
dog
dog
cat
dog
dog
cat
cat
dog
dog
dog


((17500, 200, 200, 3), (17500, 2), (7500, 200, 200, 3), (7500, 2))

From this part to the end, everything is same as before. We train 3 models and save the tensorboard results.

In [4]:
model_vgg19 = tf.keras.applications.VGG19(weights="imagenet", include_top=False, input_shape=(200, 200, 3))
model_vgg19.trainable = False
model_vgg19.summary()

chosen_optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

final_model = tf.keras.models.Sequential()
final_model.add(model_vgg19)
final_model.add(tf.keras.layers.Flatten())
final_model.add(tf.keras.layers.Dense(128, activation='relu'))
final_model.add(tf.keras.layers.Dense(64, activation='relu'))
final_model.add(tf.keras.layers.Dense(32, activation='relu'))
final_model.add(tf.keras.layers.Dense(2, activation='softmax'))

final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(training_imgs, training_labels, batch_size=32, epochs=3, validation_data=(validation_imgs, validation_labels), callbacks=tf.keras.callbacks.TensorBoard(log_dir="VGG19_Freezed_Self_In", histogram_freq=1))

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0     

In [5]:
final_model.layers[0].trainable = True
final_model.summary()
final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(training_imgs, training_labels, batch_size=32, epochs=10, validation_data=(validation_imgs, validation_labels), callbacks=tf.keras.callbacks.TensorBoard(log_dir="VGG19_Unfreezed_Self_In", histogram_freq=1))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 6, 6, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2359424   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 22,394,210
Trainable params: 22,394,210
Non-trainable params: 0
____________________________________________

In [5]:
model_mobilenetv2 = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=False, input_shape=(200, 200, 3))
model_mobilenetv2.trainable = False
model_mobilenetv2.summary()

chosen_optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

final_model = tf.keras.models.Sequential()
final_model.add(model_mobilenetv2)
final_model.add(tf.keras.layers.Flatten())
final_model.add(tf.keras.layers.Dense(128, activation='relu'))
final_model.add(tf.keras.layers.Dense(64, activation='relu'))
final_model.add(tf.keras.layers.Dense(32, activation='relu'))
final_model.add(tf.keras.layers.Dense(2, activation='softmax'))

final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(training_imgs, training_labels, batch_size=32, epochs=3, validation_data=(validation_imgs, validation_labels), callbacks=tf.keras.callbacks.TensorBoard(log_dir="MobileNetV2_Freezed_Self_In", histogram_freq=1))

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 201, 201, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 100, 100, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 100, 100, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [6]:
final_model.layers[0].trainable = True
final_model.summary()
final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(training_imgs, training_labels, batch_size=32, epochs=10, validation_data=(validation_imgs, validation_labels), callbacks=tf.keras.callbacks.TensorBoard(log_dir="MobileNetV2_Unfreezed_Self_In", histogram_freq=1))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
flatten (Flatten)            (None, 62720)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               8028288   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 10,296,674
Trainable params: 10,262,562
Non-trainable params: 34,112
_______________________________________

In [7]:
model_resnet50 = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=(200, 200, 3))
model_resnet50.trainable = False
model_resnet50.summary()

chosen_optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

final_model = tf.keras.models.Sequential()
final_model.add(model_resnet50)
final_model.add(tf.keras.layers.Flatten())
final_model.add(tf.keras.layers.Dense(128, activation='relu'))
final_model.add(tf.keras.layers.Dense(64, activation='relu'))
final_model.add(tf.keras.layers.Dense(32, activation='relu'))
final_model.add(tf.keras.layers.Dense(2, activation='softmax'))

final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(training_imgs, training_labels, batch_size=32, epochs=3, validation_data=(validation_imgs, validation_labels), callbacks=tf.keras.callbacks.TensorBoard(log_dir="ResNet50_Freezed_Self_In", histogram_freq=1))

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [8]:
final_model.layers[0].trainable = True
final_model.summary()
final_model.compile(optimizer=chosen_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

final_model.fit(training_imgs, training_labels, batch_size=32, epochs=10, validation_data=(validation_imgs, validation_labels), callbacks=tf.keras.callbacks.TensorBoard(log_dir="ResNet50_Unfreezed_Self_In", histogram_freq=1))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               12845184  
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 66        
Total params: 36,443,298
Trainable params: 36,390,178
Non-trainable params: 53,120
_____________________________________

All the results can be nicely seen using tensorboard.

In [9]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.
